# Importancia de la Interpretación de Modelos

Interpretar un modelo es fundamental por varias razones:

- **Transparencia:** Permite entender cómo y por qué el modelo toma ciertas decisiones, lo que es esencial para generar confianza en aplicaciones críticas.
- **Diagnóstico:** Facilita la identificación de problemas, como sesgos en los datos o errores en la selección de características.
- **Mejora Continua:** Al comprender la relevancia de cada variable, se pueden ajustar mejor las características y el modelo en iteraciones futuras.
- **Regulación y Ética:** En sectores regulados, es importante poder explicar las decisiones automatizadas a reguladores y usuarios.

---

# Interpretación de un Random Forest

Un Random Forest es un modelo de conjunto que combina múltiples árboles de decisión para mejorar la precisión y reducir el riesgo de sobreajuste. Aunque es un modelo de caja negra en cierta medida, existen técnicas que permiten interpretarlo:

1. **Importancia de las Variables:**
   - **Feature Importance:**  
     Cada árbol contribuye a la predicción, y se puede calcular la importancia relativa de cada variable. Esto ayuda a identificar cuáles son las variables que más influyen en el modelo.
   - **Visualización:**  
     Se pueden graficar estas importancias para ver de forma intuitiva cuáles son los factores más determinantes.

2. **Interpretación Local:**
   - **SHAP o LIME:**  
     Estas técnicas explican las predicciones a nivel individual, mostrando cómo cada característica contribuye a la predicción para un caso específico.
   
3. **Análisis de Árboles Individuales:**
   - Aunque el modelo final es un conjunto, es posible inspeccionar árboles individuales para comprender la lógica detrás de algunas decisiones.
   - Esto puede ser útil para detectar patrones o posibles errores en la selección de variables.

4. **Partial Dependence Plots:**
   - Permiten visualizar la relación entre una o más variables y la respuesta del modelo, mostrando cómo varía la predicción al cambiar el valor de una variable, mientras se mantienen constantes las demás.

En resumen, la interpretación de un Random Forest se basa en analizar la **importancia de las variables** y, opcionalmente, utilizar técnicas avanzadas para entender el impacto de cada característica en las predicciones individuales. Esto no solo ayuda a comprender el comportamiento del modelo, sino que también guía la toma de decisiones y mejora el proceso de modelado.

In [4]:
import joblib
import pandas as pd
import plotly.express as px
from pathlib import Path

In [5]:
DATA_DIR = Path.cwd() / "Modelos"

modelo_titanic = joblib.load(DATA_DIR / "titanic_classification-random_forest-v1.joblib")

# Importancia de características

Para acceder al preprocesador y al clasificador se debe:

In [6]:
preprocessor = modelo_titanic.named_steps["preprocessing"]
classifier = modelo_titanic.named_steps["classifier"]

In [7]:
preprocessor

ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['age', 'fare', 'sibsp', 'parch']),
                                ('categoric',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OneHotEncoder())]),
                                 ['sex', 'embarked']),
                                ('categoric ordinal',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OrdinalEncoder())]),
                                 ['pclass'])])

In [8]:
classifier

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=42)

In [9]:
# Acceder al preprocesador y al clasificador usando 'named_steps'
features = preprocessor.get_feature_names_out()
importances = classifier.feature_importances_

# Crear y ordenar el DataFrame con las importancias
dfFeatures = pd.DataFrame({"Variables": features, "Importancia": importances})
dfFeatures_sorted = dfFeatures.sort_values(by="Importancia", ascending=False)
print(dfFeatures_sorted)

                   Variables  Importancia
1              numeric__fare     0.212875
0               numeric__age     0.192473
4      categoric__sex_female     0.190748
5        categoric__sex_male     0.171280
9  categoric ordinal__pclass     0.125051
3             numeric__parch     0.043026
2             numeric__sibsp     0.035322
6      categoric__embarked_C     0.014530
8      categoric__embarked_S     0.010239
7      categoric__embarked_Q     0.004457


In [10]:
fig = px.bar(
    dfFeatures_sorted,
    x="Variables",
    y="Importancia",
    title="Importancia de las Variables",
    # text="Importancia"
)

fig.show()

/Users/david.palacio/Documents/academia/data-projects-lab/.venv/lib/python3.11/site-packages/kaleido/_sync_server.py:11: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




## Conclusión

- **Características Principales:**  
  - La tarifa (**fare**), el género (tanto **sex_female** como **sex_male**) y la edad son los factores más determinantes en el modelo.
  - La clase del pasajero (**pclass**) también juega un rol importante, reflejando diferencias socioeconómicas y de acceso a recursos.

- **Características Menores:**  
  - Las variables relacionadas con el número de familiares a bordo (**parch** y **sibsp**) y el puerto de embarque tienen menor impacto, lo que sugiere que, aunque pueden aportar algo de información, su influencia es relativamente pequeña.